reference:  
https://yolo-kiyoshi.com/2019/02/10/post-1075/

In [81]:
import pandas as pd
import numpy as np

In [82]:
items = pd.read_csv('../data/input/items.csv')
item_categories = pd.read_csv('../data/input/item_categories.csv')
shops = pd.read_csv('../data/input/shops.csv')
sales_train = pd.read_csv('../data/input/sales_train.csv')
test = pd.read_csv('../data/input/test.csv')

In [83]:
# 異常値(外れ値、負値)の処理
sales_train = sales_train[sales_train.item_cnt_day < 100000]
sales_train = sales_train[sales_train.item_price < 1250]

median = sales_train[(sales_train.shop_id==32)&\
                     (sales_train.item_id==2973)&\
                     (sales_train.date_block_num==4)&\
                     (sales_train.item_price>0)].item_price.median()
sales_train.loc[sales_train.item_price < 0, 'item_price'] = median

In [84]:
display(sales_train)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0
4,15.01.2013,0,25,2555,1099.0,1.0
5,10.01.2013,0,25,2564,349.0,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0


In [85]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format="%d.%m.%Y")
sales_train['year'] = sales_train.date.dt.year
sales_train['month'] = sales_train.date.dt.month
sales_train['day'] = sales_train.date.dt.day
display(sales_train)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day
0,2013-01-02,0,59,22154,999.0,1.0,2013,1,2
1,2013-01-03,0,25,2552,899.0,1.0,2013,1,3
2,2013-01-05,0,25,2552,899.0,-1.0,2013,1,5
4,2013-01-15,0,25,2555,1099.0,1.0,2013,1,15
5,2013-01-10,0,25,2564,349.0,1.0,2013,1,10
...,...,...,...,...,...,...,...,...,...
2935844,2015-10-10,33,25,7409,299.0,1.0,2015,10,10
2935845,2015-10-09,33,25,7460,299.0,1.0,2015,10,9
2935846,2015-10-14,33,25,7459,349.0,1.0,2015,10,14
2935847,2015-10-22,33,25,7440,299.0,1.0,2015,10,22


In [86]:
shops['shop_city_name'] = shops['shop_name'].apply(lambda x:x.split(' ')[0])
shops.loc[shops['shop_city_name']=='!Якутск', 'shop_city_name'] = 'Якутск'

In [87]:
# item_categoriesに大分類を追加
item_categories['major_item_category'] = \
item_categories['item_category_name'].apply(lambda x:x.split(' - ')[0])

# 中分類を追加
item_categories['sub_item_category'] = item_categories['major_item_category']

# 大分類の類似項目を集約
item_categories.loc[item_categories['major_item_category'] == 'Игровые консоли',\
                   'major_item_category'] = 'Игры'
item_categories.loc[item_categories['major_item_category'] == 'Игры PC',\
                   'major_item_category'] = 'Игры'
item_categories.loc[item_categories['major_item_category'] == 'Игры MAC',\
                   'major_item_category'] = 'Игры'
item_categories.loc[item_categories['major_item_category'] == 'Игры Android',\
                   'major_item_category'] = 'Игры'
item_categories.loc[item_categories['major_item_category'] == 'Карты оплаты (Кино, Музыка, Игры)',\
                   'major_item_category'] = 'Карты оплаты'
item_categories.loc[item_categories['major_item_category'] == 'Чистые носители (шпиль)',\
                   'major_item_category'] = 'Чистые носители'
item_categories.loc[item_categories['major_item_category'] == 'Чистые носители (штучные)',\
                   'major_item_category'] = 'Чистые носители'

In [88]:
work = pd.merge(sales_train, items, on='item_id', how='left')
work = pd.merge(work, item_categories, on='item_category_id', how='left')
work = pd.merge(work, shops, on='shop_id', how='left')
test = pd.merge(test, items, on='item_id', how='left')
test = pd.merge(test, item_categories, on='item_category_id', how='left')
test = pd.merge(test, shops, on='shop_id', how='left')

In [91]:
display(work)
display(test)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,year,month,day,item_name,item_category_id,item_category_name,major_item_category,sub_item_category,shop_name,shop_city_name
0,2013-01-02,0,59,22154,999.0,1.0,2013,1,2,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,Кино,Кино,"Ярославль ТЦ ""Альтаир""",Ярославль
1,2013-01-03,0,25,2552,899.0,1.0,2013,1,3,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
2,2013-01-05,0,25,2552,899.0,-1.0,2013,1,5,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
3,2013-01-15,0,25,2555,1099.0,1.0,2013,1,15,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
4,2013-01-10,0,25,2564,349.0,1.0,2013,1,10,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400656,2015-10-10,33,25,7409,299.0,1.0,2015,10,10,V/A Nu Jazz Selection (digipack),55,Музыка - CD локального производства,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
2400657,2015-10-09,33,25,7460,299.0,1.0,2015,10,9,V/A The Golden Jazz Collection 1 2CD,55,Музыка - CD локального производства,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
2400658,2015-10-14,33,25,7459,349.0,1.0,2015,10,14,V/A The Best Of The 3 Tenors,55,Музыка - CD локального производства,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва
2400659,2015-10-22,33,25,7440,299.0,1.0,2015,10,22,V/A Relax Collection Planet MP3 (mp3-CD) (jewel),57,Музыка - MP3,Музыка,Музыка,"Москва ТРК ""Атриум""",Москва


,ID,shop_id,item_id,item_name,item_category_id,item_category_name,major_item_category,sub_item_category,shop_name,shop_city_name
0,0,5,5037,"NHL 15 [PS3, русские субтитры]",19,Игры - PS3,Игры,Игры,"Вологда ТРЦ ""Мармелад""",Вологда
1,1,5,5320,ONE DIRECTION Made In The A.M.,55,Музыка - CD локального производства,Музыка,Музыка,"Вологда ТРЦ ""Мармелад""",Вологда
2,2,5,5233,"Need for Speed Rivals (Essentials) [PS3, русск...",19,Игры - PS3,Игры,Игры,"Вологда ТРЦ ""Мармелад""",Вологда
3,3,5,5232,"Need for Speed Rivals (Classics) [Xbox 360, ру...",23,Игры - XBOX 360,Игры,Игры,"Вологда ТРЦ ""Мармелад""",Вологда
4,4,5,5268,"Need for Speed [PS4, русская версия]",20,Игры - PS4,Игры,Игры,"Вологда ТРЦ ""Мармелад""",Вологда
...,...,...,...,...,...,...,...,...,...,...
214195,214195,45,18454,СБ. Союз 55,55,Музыка - CD локального производства,Музыка,Музыка,"Самара ТЦ ""ПаркХаус""",Самара
214196,214196,45,16188,Настольная игра Нано Кёрлинг,64,Подарки - Настольные игры,Подарки,Подарки,"Самара ТЦ ""ПаркХаус""",Самара
214197,214197,45,15757,НОВИКОВ АЛЕКСАНДР Новая коллекция,55,Музыка - CD локального производства,Музыка,Музыка,"Самара ТЦ ""ПаркХаус""",Самара
214198,214198,45,19648,ТЕРЕМ - ТЕРЕМОК сб.м/ф (Регион),40,Кино - DVD,Кино,Кино,"Самара ТЦ ""ПаркХаус""",Самара


In [93]:
pred = work[work['month']==12].groupby(['shop_id', 'item_id'], as_index=False).mean()\
[['shop_id', 'item_id', 'item_cnt_day']]
sub = pd.merge(test, pred, on=['shop_id', 'item_id'], how='left')
sub = sub[['ID', 'shop_id', 'item_id', 'item_cnt_day']]
sub.loc[sub['item_cnt_day'].isnull(), 'item_cnt_day'] = 0
sub = sub.rename(columns={'item_cnt_day':'item_cnt_month'})
sub[['ID', 'item_cnt_month']].to_csv('../data/output/output.csv', index=False, header=True)